# MNIST digit classification

We'll train a simple CNN on the MNIST dataset by copy/pasting [this example](https://keras.io/examples/mnist_cnn/) from the Keras documentation.

First, we load the data:

In [1]:
import ethik

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.
/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/max/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Model creation

Now let's create the CNN:

In [2]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W1001 16:35:56.148334 140542907729728 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1001 16:35:56.159496 140542907729728 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1001 16:35:56.161484 140542907729728 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1001 16:35:56.186977 140542907729728 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1001 16:35:56.1905

In [3]:
def train(model):
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    return model.evaluate(x_test, y_test, verbose=0)
    
def predict(model):
    return model.predict_proba(x_test)

Let's train the model and predict the class probabilities for the test set. Training and prediction steps take a lot of time so we use caching:

In [4]:
# loss, accuracy = train(model)
# y_pred = predict(model)
# np.save("cache/mnist.npy", y_pred)

y_pred = np.load("cache/mnist.npy")

`y_pred` is a `(n_samples, n_features)` (i.e. `(n_samples, n_digits)`) numpy array of probabilities:

In [5]:
y_pred.shape

(10000, 10)

In [6]:
np.isclose(np.sum(y_pred[0]), 1)

True

# Bias explanation

Now, we have the data and can use `ethik` to explain it. There is one feature per pixel:

In [7]:
explainer = ethik.ImageClassificationExplainer()
fig = explainer.plot_bias(x_test, y_pred)


fig.write_image("mnist_bias_explanation.svg")

100%|██████████| 14520/14520 [00:17<00:00, 848.11it/s]


The previous plot highlights the regions of importance for identifying each digit. More precisely, the intensity of each pixel corresponds to the probability increase of saturating or not the pixel. A value of 0.28 means that saturating the pixel increases the probability predicted by the model by 0.28. Note that we do not saturate and desaturate the pixels independently. Instead, our method understands which pixels are linked together and saturates them in a realistic manner. The previous images show that the CNN seems to be using the same visual cues as a human. However, we can see that is uses very specific regions on images to identify particular digits. For instance, the top-right region of an image seems to trigger the "5" digit, whereas the bottom parts of the images seem to be linked with the "7" digit.

# Performance explanation

`y_test` is an array of `n_samples` one-hot-encoded vectors:

In [8]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

To get the label, we use `.argmax()`:

In [9]:
y_test.argmax(axis=1)

array([7, 2, 1, ..., 4, 5, 6])

Then we can explain the model performance with `ethik`. Because of `sklearn.metrics.accuracy_score()` API, we need to convert the prediction vectors into their corresponding label:

In [10]:
from sklearn import metrics

explainer.plot_performance(
    X_test=x_test,
    y_test=y_test.argmax(axis=1),
    y_pred=y_pred.argmax(axis=1),
    metric=metrics.accuracy_score
)

100%|██████████| 1452/1452 [00:02<00:00, 605.21it/s]


**TODO: analysis**

The log loss metric deals with vectors of probabilities so we don't need to get the label:

In [11]:
explainer.plot_performance(
    X_test=x_test,
    y_test=y_test,
    y_pred=y_pred,
    metric=metrics.log_loss
)

100%|██████████| 1452/1452 [00:14<00:00, 103.25it/s]


**TODO: analysis and comparison with `accuracy_score()`**